In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

tf.config.list_physical_devices('GPU')

TensorFlow version: 2.18.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [24]:
class CNN(Model):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.fcn_1 = Dense(128, activation="relu")
        self.fcn_2 = Dense(10)

    def call(self, x):
        x = self.conv_1(x)
        x = self.flatten(x)
        x = self.fcn_1(x)
        x = self.fcn_2(x)
        return x

    def get_config(self):
        """
        Return the configuration of the model.
        This is optional but recommended if you want Keras to be able
        to clone and serialize this model cleanly.
        """
        base_config = super().get_config()
        # If you had custom arguments to your layers, you'd return them here as well.
        return {
            **base_config,
        }

model = CNN()

In [25]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [26]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [27]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [28]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [29]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result():0.2f}, '
    f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
    f'Test Loss: {test_loss.result():0.2f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
  )

Epoch 1, Loss: 0.13, Accuracy: 96.11, Test Loss: 0.06, Test Accuracy: 98.02
Epoch 2, Loss: 0.04, Accuracy: 98.75, Test Loss: 0.05, Test Accuracy: 98.42
Epoch 3, Loss: 0.02, Accuracy: 99.34, Test Loss: 0.06, Test Accuracy: 98.24
Epoch 4, Loss: 0.01, Accuracy: 99.62, Test Loss: 0.06, Test Accuracy: 98.36
Epoch 5, Loss: 0.01, Accuracy: 99.72, Test Loss: 0.06, Test Accuracy: 98.35


In [30]:
model.save("tf_mnist_clf.keras")

In [33]:
new_model = tf.keras.models.load_model(
    "tf_mnist_clf.keras",
    custom_objects={"CNN": CNN}  # Only needed if TensorFlow cannot automatically detect the custom class
)

new_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [34]:
# Evaluate the loaded model on the test data
test_loss, test_accuracy = new_model.evaluate(x_test, y_test, verbose=2)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Make predictions on a small batch of test images (e.g., first 5 images)
predictions = new_model.predict(x_test[:5])
print("Predictions shape:", predictions.shape)
print("Predictions (logits) for first 5 test samples:\n", predictions)

# Optionally, you can see which class has the highest logit for the first sample
import numpy as np
predicted_labels = np.argmax(predictions, axis=1)
print("Predicted labels for first 5 test samples:", predicted_labels)
print("True labels for first 5 test samples:", y_test[:5])

313/313 - 2s - 7ms/step - accuracy: 0.9835 - loss: 0.0572
Test Loss: 0.0572, Test Accuracy: 0.9835
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step
Predictions shape: (5, 10)
Predictions (logits) for first 5 test samples:
 [[ -8.960968    -3.6098053   -1.9047667    3.8425546  -16.239109
   -7.246529   -23.59654     18.997494    -4.2936597    0.82871115]
 [ -2.0110133    3.7466779   24.194775    -3.6799045  -10.3846655
  -15.953404    -3.0973895  -12.947412    -7.850632    -8.3854475 ]
 [ -5.1603994   12.16938     -5.5844026  -14.708157     1.2995207
   -4.4629374   -5.1674304   -2.074558     0.31868565  -7.479786  ]
 [ 19.675869   -12.392718    -0.98889226  -9.754522    -6.913312
   -8.05639      1.7450628   -3.2789075   -2.6150756   -2.8733907 ]
 [ -3.360126    -8.127602    -6.508153    -9.097588    19.347935
   -3.2268329   -3.5319448   -1.4711944   -2.4427676    8.955972  ]]
Predicted labels for first 5 test samples: [7 2 1 0 4]
True labels for first 5 test samples: [7 2 1 0 4]
